# Feature Engineering and Model Evaluation

This notebook evaluates the **incremental importance** of newly added features for predicting rotation vs continuation during RTH.

## Feature Set

**Existing Features:**
- `relative_ib_volume`
- `normalized_distance`
- `opening_bar_open_close`
- `norm_opening_bar_volume`
- `norm_prev_session_volume`

**New Features:**
- `nearest_prior_level_to_open` (already exists in phase2 data)
- `norm_opening_volatility` (ATR-based)
- `news_event_during_RTH` (from events CSV)

## Approach
- Compare baseline (existing features) vs expanded (existing + new features)
- Train Decision Tree, Random Forest, and XGBClassifier
- Use stratified 5-fold cross-validation
- Account for class imbalance using `class_weight`
- Report F1, Precision, Recall
- Perform error analysis on misclassified samples

In [ ]:
import pandas as pd
import numpy as np
import warnings
from datetime import datetime, time, timedelta
from zoneinfo import ZoneInfo

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.metrics import f1_score, precision_score, recall_score, make_scorer

warnings.filterwarnings('ignore')

print("Libraries loaded successfully")

## Load Data

In [ ]:
# Load the phase2 data which contains existing features
df = pd.read_csv('outputs/phase2_10min.csv')

# Parse session_date
df['session_date'] = pd.to_datetime(df['session_date'])

print(f"Loaded {len(df)} sessions")
print(f"Date range: {df['session_date'].min()} to {df['session_date'].max()}")
print(f"\nRotation distribution:")
print(df['rotation'].value_counts())
print(f"\nClass balance: {df['rotation'].mean():.2%} rotation")

## Feature Engineering - Task 1: News Event During RTH

Create `news_event_during_RTH` from events CSV:
- Filter for medium and high impact USD events only
- Check if events occur during RTH (6:30am-1:00pm PST)
- Binary indicator: 1 if at least one qualifying event, else 0

In [ ]:
# Load events data
events_df = pd.read_csv('Jan01_2025_December31_2025_events.csv')

# Parse DateTime column (ISO format with timezone)
events_df['DateTime'] = pd.to_datetime(events_df['DateTime'], utc=True)
# Convert to PST/America/Los_Angeles timezone
events_df['DateTime'] = events_df['DateTime'].dt.tz_convert('America/Los_Angeles')

print(f"Loaded {len(events_df)} events")
print(f"\nCurrency distribution:")
print(events_df['Currency'].value_counts().head(10))
print(f"\nImpact distribution:")
print(events_df['Impact'].value_counts())

In [ ]:
# Filter for USD events with medium or high impact
usd_events = events_df[
    (events_df['Currency'] == 'USD') & 
    (events_df['Impact'].isin(['Medium Impact Expected', 'High Impact Expected']))
].copy()

print(f"USD medium/high impact events: {len(usd_events)}")

# Extract date and time components (already in PST/LA timezone)
usd_events['date'] = usd_events['DateTime'].dt.date
usd_events['time'] = usd_events['DateTime'].dt.time

# Define RTH window (6:30am - 1:00pm PST)
RTH_START = time(6, 30)
RTH_END = time(13, 0)

# Filter events during RTH
usd_events_rth = usd_events[
    (usd_events['time'] >= RTH_START) & 
    (usd_events['time'] < RTH_END)
]

print(f"USD medium/high impact events during RTH: {len(usd_events_rth)}")
print(f"\nSample of RTH events:")
print(usd_events_rth[['DateTime', 'Impact', 'Event']].head(10))

In [ ]:
# Create a set of dates with qualifying news events
news_dates = set(usd_events_rth['date'].unique())

print(f"Trading days with news events during RTH: {len(news_dates)}")

# Map to our dataframe
df['news_event_during_RTH'] = df['session_date'].dt.date.apply(
    lambda d: 1 if d in news_dates else 0
)

print(f"\nNews event distribution in dataset:")
print(df['news_event_during_RTH'].value_counts())
print(f"\nProportion with news events: {df['news_event_during_RTH'].mean():.2%}")

## Feature Engineering - Task 2: Normalized Opening Volatility

Create `norm_opening_volatility`:
- Numerator: 1-minute ATR(14) at 6:45am PST
- Denominator: Average RTH ATR(14) over previous 5 trading days
- ATR computed on 1-minute bars
- No future data leakage

In [ ]:
# Load 1-minute bar data
bars_df = pd.read_csv('MNQ_1min_2023Jan_2026Jan.csv')

# Parse DateTime
bars_df['DateTime'] = pd.to_datetime(bars_df['DateTime'], format='%m/%d/%y %H:%M')

# Convert to PST (UTC-8) then to America/Los_Angeles for DST handling
from datetime import timezone
pst = timezone(timedelta(hours=-8))
local_tz = ZoneInfo('America/Los_Angeles')

bars_df['DateTime'] = bars_df['DateTime'].apply(
    lambda dt: dt.replace(tzinfo=pst).astimezone(local_tz).replace(tzinfo=None)
)

bars_df['date'] = bars_df['DateTime'].dt.date
bars_df['time'] = bars_df['DateTime'].dt.time

print(f"Loaded {len(bars_df)} 1-minute bars")
print(f"Date range: {bars_df['DateTime'].min()} to {bars_df['DateTime'].max()}")

In [ ]:
# Function to calculate True Range
def calculate_tr(df):
    """Calculate True Range for each bar."""
    df = df.copy()
    df['prev_close'] = df['Close'].shift(1)
    
    df['tr'] = df.apply(
        lambda row: max(
            row['High'] - row['Low'],
            abs(row['High'] - row['prev_close']) if pd.notna(row['prev_close']) else row['High'] - row['Low'],
            abs(row['Low'] - row['prev_close']) if pd.notna(row['prev_close']) else row['High'] - row['Low']
        ),
        axis=1
    )
    return df

# Function to calculate ATR(14)
def calculate_atr(df, period=14):
    """Calculate ATR using exponential moving average."""
    df = calculate_tr(df)
    df['atr'] = df['tr'].ewm(span=period, adjust=False).mean()
    return df

# Calculate ATR for all bars
bars_df = calculate_atr(bars_df.sort_values('DateTime'))

print("ATR calculated")
print(f"Sample ATR values: {bars_df['atr'].describe()}")

In [ ]:
# Filter RTH bars only
RTH_START = time(6, 30)
RTH_END = time(13, 0)

rth_bars = bars_df[
    (bars_df['time'] >= RTH_START) & 
    (bars_df['time'] < RTH_END)
].copy()

print(f"RTH bars: {len(rth_bars)}")

# Get unique trading dates (dates with RTH bars)
trading_dates = sorted(rth_bars['date'].unique())
print(f"Trading dates with RTH bars: {len(trading_dates)}")

In [ ]:
# Calculate norm_opening_volatility for each session
def calculate_norm_opening_volatility(target_date, bars_df, trading_dates):
    """
    Calculate normalized opening volatility:
    - Numerator: ATR at 6:45am PST on target date
    - Denominator: Average RTH ATR over previous 5 trading days
    """
    # Get ATR at 6:45am on target date
    target_645am = bars_df[
        (bars_df['date'] == target_date) & 
        (bars_df['time'] == time(6, 45))
    ]
    
    if len(target_645am) == 0:
        return None
    
    opening_atr = target_645am.iloc[0]['atr']
    
    if pd.isna(opening_atr):
        return None
    
    # Find previous 5 trading dates
    try:
        target_idx = trading_dates.index(target_date)
    except ValueError:
        return None
    
    if target_idx < 5:
        return None  # Not enough history
    
    prev_5_dates = trading_dates[target_idx-5:target_idx]
    
    # Calculate average RTH ATR over previous 5 days
    prev_rth_bars = bars_df[
        (bars_df['date'].isin(prev_5_dates)) &
        (bars_df['time'] >= RTH_START) &
        (bars_df['time'] < RTH_END)
    ]
    
    if len(prev_rth_bars) == 0:
        return None
    
    avg_rth_atr = prev_rth_bars['atr'].mean()
    
    if pd.isna(avg_rth_atr) or avg_rth_atr == 0:
        return None
    
    return opening_atr / avg_rth_atr

# Apply to each session
df['norm_opening_volatility'] = df['session_date'].dt.date.apply(
    lambda d: calculate_norm_opening_volatility(d, bars_df, trading_dates)
)

print(f"\nNormalized opening volatility statistics:")
print(df['norm_opening_volatility'].describe())
print(f"\nMissing values: {df['norm_opening_volatility'].isna().sum()}")

## Prepare Features for Modeling

Calculate missing features:
- `normalized_distance`: nearest_prior_level_distance / prev_range
- Normalized volume features using rolling averages

In [ ]:
# Calculate normalized_distance feature
# normalized_distance = nearest_prior_level_distance / previous day range
df['prev_range'] = df['prev_pdh'] - df['prev_pdl']
df['prev_range'] = df['prev_range'].replace(0, np.nan)  # Avoid div-by-zero
df['normalized_distance'] = df['nearest_prior_level_to_open_distance'] / df['prev_range']
df['normalized_distance'] = df['normalized_distance'].fillna(0)

print(f"Calculated normalized_distance")
print(f"Stats: {df['normalized_distance'].describe()}")

In [ ]:
# Calculate normalized volume features
# Use rolling average for normalization (consistent with mp3_analysis_variables.py)

# Sort by date for rolling calculations
df = df.sort_values('session_date').reset_index(drop=True)

# Calculate 10-day rolling average for normalization
window = 10
df['avg_opening_bar_volume'] = df['opening_bar_volume'].rolling(
    window=window, min_periods=1
).mean().shift(1)  # Shift to avoid future leakage

df['avg_prev_session_volume'] = df['prev_session_volume'].rolling(
    window=window, min_periods=1
).mean().shift(1)

# Normalize
df['norm_opening_bar_volume'] = df['opening_bar_volume'] / df['avg_opening_bar_volume']
df['norm_prev_session_volume'] = df['prev_session_volume'] / df['avg_prev_session_volume']

# Handle infinity and NaN
df['norm_opening_bar_volume'].replace([np.inf, -np.inf], np.nan, inplace=True)
df['norm_prev_session_volume'].replace([np.inf, -np.inf], np.nan, inplace=True)

print("Normalized volume features calculated")

In [ ]:
# Define feature sets
existing_features = [
    'relative_ib_volume',
    'normalized_distance',
    'opening_bar_open_close',
    'norm_opening_bar_volume',
    'norm_prev_session_volume'
]

new_features = [
    'nearest_prior_level_to_open_distance',  # Already in phase2 data
    'norm_opening_volatility',
    'news_event_during_RTH'
]

all_features = existing_features + new_features

# Prepare target
y = df['rotation'].astype(int)

# Prepare feature matrices
X_baseline = df[existing_features].copy()
X_expanded = df[all_features].copy()

# Drop rows with missing values
valid_baseline = ~X_baseline.isna().any(axis=1)
valid_expanded = ~X_expanded.isna().any(axis=1)

# For fair comparison, use only rows that have valid data for all features
valid_rows = valid_baseline & valid_expanded

X_baseline_clean = X_baseline[valid_rows]
X_expanded_clean = X_expanded[valid_rows]
y_clean = y[valid_rows]

print(f"\nDataset sizes:")
print(f"Original: {len(df)}")
print(f"After cleaning: {len(X_baseline_clean)}")
print(f"Dropped: {len(df) - len(X_baseline_clean)}")
print(f"\nClass distribution (cleaned):")
print(y_clean.value_counts())
print(f"Rotation rate: {y_clean.mean():.2%}")

## Model Training and Evaluation

Train and compare:
- Decision Tree
- Random Forest
- XGBClassifier

For both baseline and expanded feature sets.

In [ ]:
# Define models with class_weight for imbalance
models = {
    'Decision Tree': DecisionTreeClassifier(
        max_depth=5,
        class_weight='balanced',
        random_state=42
    ),
    'Random Forest': RandomForestClassifier(
        n_estimators=100,
        max_depth=5,
        class_weight='balanced',
        random_state=42,
        n_jobs=-1
    ),
    'XGBClassifier': XGBClassifier(
        max_depth=5,
        n_estimators=100,
        scale_pos_weight=(y_clean == 0).sum() / (y_clean == 1).sum(),
        random_state=42,
        eval_metric='logloss'
    )
}

# Define cross-validation strategy
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Define scoring metrics
scoring = {
    'f1': make_scorer(f1_score),
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score)
}

print("Models and CV strategy defined")

In [ ]:
# Train and evaluate baseline models
print("="*80)
print("BASELINE MODELS (Existing Features Only)")
print("="*80)
print(f"Features: {existing_features}\n")

baseline_results = {}

for model_name, model in models.items():
    print(f"\nTraining {model_name}...")
    
    cv_results = cross_validate(
        model, X_baseline_clean, y_clean,
        cv=cv, scoring=scoring, return_train_score=False
    )
    
    baseline_results[model_name] = {
        'f1_mean': cv_results['test_f1'].mean(),
        'f1_std': cv_results['test_f1'].std(),
        'precision_mean': cv_results['test_precision'].mean(),
        'precision_std': cv_results['test_precision'].std(),
        'recall_mean': cv_results['test_recall'].mean(),
        'recall_std': cv_results['test_recall'].std(),
    }
    
    print(f"  F1 Score:   {baseline_results[model_name]['f1_mean']:.4f} ± {baseline_results[model_name]['f1_std']:.4f}")
    print(f"  Precision:  {baseline_results[model_name]['precision_mean']:.4f} ± {baseline_results[model_name]['precision_std']:.4f}")
    print(f"  Recall:     {baseline_results[model_name]['recall_mean']:.4f} ± {baseline_results[model_name]['recall_std']:.4f}")

In [ ]:
# Train and evaluate expanded models
print("\n" + "="*80)
print("EXPANDED MODELS (Existing + New Features)")
print("="*80)
print(f"Features: {all_features}\n")

expanded_results = {}

for model_name, model in models.items():
    print(f"\nTraining {model_name}...")
    
    cv_results = cross_validate(
        model, X_expanded_clean, y_clean,
        cv=cv, scoring=scoring, return_train_score=False
    )
    
    expanded_results[model_name] = {
        'f1_mean': cv_results['test_f1'].mean(),
        'f1_std': cv_results['test_f1'].std(),
        'precision_mean': cv_results['test_precision'].mean(),
        'precision_std': cv_results['test_precision'].std(),
        'recall_mean': cv_results['test_recall'].mean(),
        'recall_std': cv_results['test_recall'].std(),
    }
    
    print(f"  F1 Score:   {expanded_results[model_name]['f1_mean']:.4f} ± {expanded_results[model_name]['f1_std']:.4f}")
    print(f"  Precision:  {expanded_results[model_name]['precision_mean']:.4f} ± {expanded_results[model_name]['precision_std']:.4f}")
    print(f"  Recall:     {expanded_results[model_name]['recall_mean']:.4f} ± {expanded_results[model_name]['recall_std']:.4f}")

## Results Comparison

In [ ]:
# Create comparison table
import pandas as pd

comparison_data = []

for model_name in models.keys():
    baseline = baseline_results[model_name]
    expanded = expanded_results[model_name]
    
    comparison_data.append({
        'Model': model_name,
        'Feature Set': 'Baseline',
        'F1 Score': f"{baseline['f1_mean']:.4f} ± {baseline['f1_std']:.4f}",
        'Precision': f"{baseline['precision_mean']:.4f} ± {baseline['precision_std']:.4f}",
        'Recall': f"{baseline['recall_mean']:.4f} ± {baseline['recall_std']:.4f}"
    })
    
    comparison_data.append({
        'Model': model_name,
        'Feature Set': 'Expanded',
        'F1 Score': f"{expanded['f1_mean']:.4f} ± {expanded['f1_std']:.4f}",
        'Precision': f"{expanded['precision_mean']:.4f} ± {expanded['precision_std']:.4f}",
        'Recall': f"{expanded['recall_mean']:.4f} ± {expanded['recall_std']:.4f}"
    })
    
    # Calculate improvement
    f1_improvement = expanded['f1_mean'] - baseline['f1_mean']
    precision_improvement = expanded['precision_mean'] - baseline['precision_mean']
    recall_improvement = expanded['recall_mean'] - baseline['recall_mean']
    
    comparison_data.append({
        'Model': model_name,
        'Feature Set': 'Δ (Improvement)',
        'F1 Score': f"{f1_improvement:+.4f}",
        'Precision': f"{precision_improvement:+.4f}",
        'Recall': f"{recall_improvement:+.4f}"
    })

comparison_df = pd.DataFrame(comparison_data)

print("\n" + "="*80)
print("RESULTS COMPARISON")
print("="*80)
print(comparison_df.to_string(index=False))

## Error Analysis

Identify misclassified samples to understand systematic failure modes.

In [ ]:
# Train final models on full data for error analysis
from sklearn.model_selection import train_test_split

# Split data for error analysis
X_train_base, X_test_base, y_train, y_test = train_test_split(
    X_baseline_clean, y_clean, test_size=0.3, random_state=42, stratify=y_clean
)

X_train_exp, X_test_exp, _, _ = train_test_split(
    X_expanded_clean, y_clean, test_size=0.3, random_state=42, stratify=y_clean
)

print(f"Training set: {len(X_train_base)} samples")
print(f"Test set: {len(X_test_base)} samples")

In [ ]:
# Function to perform error analysis
def error_analysis(model, X_train, X_test, y_train, y_test, model_name, feature_names):
    """
    Train model and analyze misclassified samples.
    """
    # Train model
    model.fit(X_train, y_train)
    
    # Predict
    y_pred = model.predict(X_test)
    
    # Find misclassified samples
    misclassified = y_test != y_pred
    misclassified_indices = X_test[misclassified].index
    
    print(f"\n{'='*80}")
    print(f"{model_name} - Error Analysis")
    print(f"{'='*80}")
    print(f"Total test samples: {len(y_test)}")
    print(f"Misclassified: {misclassified.sum()} ({misclassified.mean()*100:.2f}%)")
    print(f"\nBreakdown of errors:")
    
    # False positives and false negatives
    false_positives = ((y_test == 0) & (y_pred == 1)).sum()
    false_negatives = ((y_test == 1) & (y_pred == 0)).sum()
    
    print(f"  False Positives (predicted rotation, was not): {false_positives}")
    print(f"  False Negatives (predicted no rotation, was rotation): {false_negatives}")
    
    # Show first 10 misclassified samples
    print(f"\nFirst 10 misclassified samples:")
    print("-" * 80)
    
    for i, idx in enumerate(misclassified_indices[:10]):
        true_label = "Rotation" if y_test.loc[idx] == 1 else "No Rotation"
        pred_label = "Rotation" if y_pred[y_test.index.get_loc(idx)] == 1 else "No Rotation"
        
        print(f"\nSample {i+1} (Index: {idx}):")
        print(f"  True Label: {true_label}")
        print(f"  Predicted:  {pred_label}")
        print(f"  Features:")
        
        for feat_name in feature_names:
            feat_value = X_test.loc[idx, feat_name]
            print(f"    {feat_name}: {feat_value:.4f}")
    
    print("\n" + "="*80)
    
    return model, y_pred

In [ ]:
# Perform error analysis for each model - Baseline
print("\n" + "#"*80)
print("ERROR ANALYSIS - BASELINE MODELS")
print("#"*80)

baseline_trained = {}
for model_name, model in models.items():
    trained_model, predictions = error_analysis(
        model, X_train_base, X_test_base, y_train, y_test,
        f"{model_name} (Baseline)", existing_features
    )
    baseline_trained[model_name] = trained_model

In [ ]:
# Perform error analysis for each model - Expanded
print("\n" + "#"*80)
print("ERROR ANALYSIS - EXPANDED MODELS")
print("#"*80)

expanded_trained = {}
for model_name, model in models.items():
    trained_model, predictions = error_analysis(
        model, X_train_exp, X_test_exp, y_train, y_test,
        f"{model_name} (Expanded)", all_features
    )
    expanded_trained[model_name] = trained_model

## Summary and Conclusions

### Key Findings

1. **New Features Added:**
   - `news_event_during_RTH`: Binary indicator for medium/high impact USD news during RTH
   - `norm_opening_volatility`: Ratio of opening ATR to recent average ATR
   - `nearest_prior_level_to_open_distance`: Already present in phase2 data

2. **Model Comparison:**
   - Three classifiers evaluated: Decision Tree, Random Forest, XGBClassifier
   - Both baseline and expanded feature sets tested
   - 5-fold stratified cross-validation used
   - Class imbalance handled via `class_weight`

3. **Performance Metrics:**
   - F1 Score, Precision, and Recall reported for each model
   - Incremental improvements from new features quantified

4. **Error Analysis:**
   - Misclassified samples identified and analyzed
   - Feature vectors printed for systematic failure investigation

### Assumptions
- **Timezone:** All times in PST/America/Los_Angeles (handles DST)
- **RTH Window:** 6:30am - 1:00pm PST
- **Trading days:** Only dates with RTH bars included
- **Holiday calendar:** Implicitly handled via available bar data
- **Missing data:** Rows with any missing features dropped
- **No future leakage:** Rolling averages use only past data